In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from numpy import random

!pip install torcheval
from torcheval.metrics.functional import multiclass_f1_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 9.0 MB/s eta 0:00:00


In [2]:
df = pd.read_csv('Train_set.csv')

In [3]:
# Linearly expand
def expand_linear(time_series):
  left = 0
  right = 0
  for k in range(1, len(time_series)):
    if time_series[k] == 0:
      left = time_series[k-1]
      l = 0
      while time_series[k+l] == 0:
        l+=1
        if len(time_series) == k+l:
          return time_series
      right = time_series[k+l]
      time_series[k: k+l] = np.linspace(left, right, num=l+2)[1:(l+1)]
  return time_series

In [ ]:
# Preprocessing

In [4]:
# Simple idea of putting 0
df.fillna(0, inplace = True)
# Separate the ID and class columns from the input features
ids = df['ID']
y = df['Class']
y = np.array(y)
X = df.drop(['ID', 'Class'], axis=1)
X = np.array(X)

# Get lengths
lengths = [np.trim_zeros(X[k, :], 'b').size for k in range(len(X))]
lengths = np.array(lengths) -1
# Expand 
for k in range(X.shape[0]):
  expand_linear(X[k,:])

# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# valid set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [5]:
# Convert the preprocessed data to PyTorch tensors
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y).long()

# Create a PyTorch DataLoader to handle batching and shuffling of the data
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [6]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, lengths, output_size = 5, lr=0.005):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.attention = nn.MultiheadAttention(embed_dim = hidden_size, num_heads = 1, batch_first = True)

        self.lengths = lengths
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(), lr)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.attention(out[:,self.lengths,:], out, out)[0]
        out = self.fc(out[:, -1, :])
        return out

    def trainloop(self, num_epochs=40):

      # Initialize the model and move it to the device

      # Define the loss function and optimizer


      # Train the model
      for epoch in range(num_epochs):
          for X_batch, y_batch in dataloader:
              
              
              X_batch = X_batch.view([X_batch.shape[0],X_batch.shape[1],1]).to(self.device)
              y_batch = y_batch.to(self.device)
              running_loss = 0.
              # Zero out the gradients
              self.optimizer.zero_grad()
              self.train()
              # Forward pass
              outputs = self.forward(X_batch)
              loss = self.criterion(outputs, y_batch)
  
              # Backward pass and optimization
              loss.backward()
              self.optimizer.step()

              running_loss += loss.item()
          
          # Print the loss and validation accuracy at the end of each epoch
          with torch.no_grad():
              total_loss = 0
              total_correct = 0
              total_samples = 0
              for X_val, y_val in dataloader:
                  X_val = X_val.to(self.device)
                  y_val = y_val.to(self.device)
                  outputs = self.forward(X_val)
                  total_loss += self.criterion(outputs, y_val).item() * X_val.shape[0]
                  total_correct += (outputs.argmax(dim=1) == y_val).sum().item()
                  total_samples += X_val.shape[0]
              val_loss = total_loss / total_samples
              val_acc = total_correct / total_samples
              val_F1 = multiclass_f1_score(outputs, y_val, num_classes=5)

          print(f"Epoch {epoch+1}/{num_epochs}: train_loss={loss.item():.4f} val_loss={val_loss:.4f} val_acc={val_acc:.4f} val_F1={val_F1:.4f}")

In [ ]:
model = LSTMModel(input_size=1, hidden_size=128, num_layers=1, lengths = lengths)
model.trainloop(num_epochs = 1)

In [ ]:
# Load test data into a pandas DataFrame
test_data = pd.read_csv("Test_set.csv")

# Impute NaN values with the median value of each column
test_data.fillna(0, inplace=True)

# Drop irrelevant columns
test_data = test_data.drop(columns=["ID"])

# Expand 
for k in range(test_data.shape[0]):
  expand_linear(test_data[k,:])
# Standardize the input features
scaler = StandardScaler()
test_data = scaler.fit_transform(test_data)

# Reshape the input features into a 3D array for input to the LSTM
#test_data = test_data.reshape(test_data.shape[0], test_data.shape[1], 1)

# Convert the numpy array to a PyTorch tensor
#test_tensor = torch.Tensor(test_data)

# Create a DataLoader object for the test data
#test_loader = torch.utils.data.DataLoader(test_tensor, batch_size=32, shuffle=False)


In [ ]:
pred_classes = [torch.argmax(model.forward(test_data[k,:])) for k in range(test_data.shape[0])]
pred_classes = np.array(pred_classes)
ids = np.arange(len(pred_classes))
results = pd.DataFrame({'ID': ids, 'Pred_Class': pred_classes})

# Save the DataFrame as a CSV file
results.to_csv('sub_30-04.csv', index=False)

# 0 class
sum(pred_classes ==0)

In [ ]:
### NOT USED
# Set the model to evaluation mode
model.eval()

# Create an empty list to store the predicted classes
pred_classes = []

# Loop through the test data in batches using the test loader
for X_batch in test_loader:
    # Move the batch to the device
    #X_batch = X_batch.to(device)

    # Compute the model's predicted class for the batch
    outputs = model.forward(X_batch)
    _, predicted = torch.max(outputs.data, 1)

    # Convert the predicted class tensor to a numpy array
    predicted = predicted.numpy()

    # Append the predicted classes to the list
    pred_classes.append(predicted)

# Convert the list of predicted classes to a numpy array
pred_classes = np.concatenate(pred_classes)

# Create a new pandas DataFrame with the predicted classes and an ID column (if necessary)
ids = np.arange(len(pred_classes))
df = pd.DataFrame({'ID': ids, 'Pred_Class': pred_classes})

# Save the DataFrame as a CSV file
df.to_csv('submission1.csv', index=False)

